# Analysis Ready Data Tutorial

Time-series analysis (e.g. change detection and trend detection) is a powerful application of satellite imagery. However, a great deal of processing is required to prepare imagery for analysis. Analysis Ready Data (ARD), preprocessed time-series stacks of overhead imagery, allow for time-series analysis without any additional processing of the imagery. See [Analysis Data Defined](https://medium.com/planet-stories/analysis-ready-data-defined-5694f6f48815) for an excellent introduction and discussion on ARD.

This tutorial shows how [Planet APIs](https://developers.planet.com/docs/apis/) can simplify production of ARD by working through two real-world use cases. This tutorial is targeted to users who have little to no geospatial knowledge but have experience working with APIs. The goal of this tutorial is to teach the user the how and whys of using the Data and Orders APIs to create and interpret ARD for both use cases. The use cases addressed in this tutorial are:

* As a software engineer at an ag-tech company, I'd like to be able to order Planet imagery programmatically in a way that enables the data scientist at my organization to create time-series algorithms (e.g. monitoring ndvi curves over time) without further data cleaning and processing.
* As a tropical forest monitoring analyst, I'd like to create an imagery pipeline that delivers analysis-ready imagery over a protected area so I can create simple change detection (ndvi_xdate - ndvi_ydate) analyses. I'd like to do minimal custom (non-Planet) data processing to get the imagery in a format that supports a change-detection analysis because my technical skills are limited.

## APIs

The two Planet APIs that are used in creation of ARD are the [Data API](https://developers.planet.com/docs/data/) and the [Orders API](https://developers.planet.com/docs/orders/). The Data API is used to search for imagery according to certain criteria based on the use case, and the Orders API is used to process that imagery into ARD that can be fed directly into time-series analysis.

### Data API

The first step in using the Data API is identifying the search criteria. This is specifying answers to the following questions regarding the use case time-series analysis:
* What is the time range?
* What product item type is desired?
* What is the area of interest (geographic region)?
* What percentage of pixels need to be usable?
* etc.

While time range is likely pretty trivial to determine, product item, area of interest, and usable pixels may take a little bit of work. Let's dive into each further

#### Product Item Type

The [product item type](https://developers.planet.com/docs/data/items-assets/) refers to the sensor source (aka satellite type) and basic processing desired. This decision is highly dependent on application, as coverage, revisit rate, spectral bands, and resolution differ between products. A good overview of the products available in the Planet Data API is provided on the [Planet Imagery and Archive](https://www.planet.com/products/planet-imagery/) page (look for the link to product specs for details). For most frequent revisit rate, we will use the PS satellite. Experience has shown that customers most often use the scene (vs the orthotile) product. Therefore, this tutorial will focus on the `PSScene4Band` product.

#### Area of Interest

The area of interest is the geographic region for the analysis, given as GeoJSON. If you are familiar with JSON, the format of GeoJSON will likely be easy to grasp. According to <geojson.org>, "GeoJSON is a format for encoding a variety of geographic data structures." The specific geographic data structure we are interested is a `Polygon`. The [GeoJSON wikipedia page](https://en.wikipedia.org/wiki/GeoJSON) gives some great examples of the data structures for the various GeoJSON data structures. 

Some care needs to be given to describing the [position](https://tools.ietf.org/html/rfc7946#section-3.1.1) for each coordinate in a GeoJSON geometry. Each position is specified as `(longitude, latitude)` or `(easting, northing)` (order really matters here!). Also, this may be a surprise, but the same point on earth can have different `(longitude, latitude)` values based on the spatial reference system. Basically, a spatial reference system describes where something is in the real world. But there are thousands of different spatial reference systems. These are separated into two categories: geographic and projected. Geographic coordinate systems model the earth as an ellipsoid (this model is called the `datum`) and describe positions on the surface of the earth (coordinates) in terms of the prime meridian and angle of measure. Projected coordinate systems take this a step further by projecting the geographic coordinates (quite three-dimensional) into two dimensions (the `projection`). Different projections preserve different properties, such as area, angles, or direction for north. There is a rich area of discovery, discussion, and even a little [teasing](https://xkcd.com/977/) (thanks xkcd!) in the world of spatial reference systems.

GeoJSON only supports one spatial reference system, [WGS84](https://spatialreference.org/ref/epsg/wgs-84/). This is a geographic coordinate system describing locations in latitude, longitude. However, many web mapping applications use the Web Mercator projected coordinate system to to describe locations. Confusingly, Web Mercator also describes locations in latitude, longitude. But a `(longitude, latitude)` GeoJSON position given in Web Mercator will **not** end up where you expect if it is not first projected into WGS84. The easiest way to define an aoi that is described in WGS84 is to draw it in [Planet Explorer](https://www.planet.com/explorer) and click the little button "Download AOI as GeoJSON."

#### Usable Pixels

Taking pictures of the earth from space ain't easy. There are a lot of steps and a lot of things have to go right to get a clear shot. Therefore, not every pixel in every image taken from space is useful for analysis. For one, images taken from space are projected into a spatial reference system (discussed briefly above), a process that introduces some NoData (aka outside of the image footprint) pixels into the resulting image. Additionally, clouds cover a great deal of the earth and create cloudy pixels when imaged. While some applications can use cloudy pixels, others cannot. Therefore, the type of pixels that are determined to be 'usable' are often application-specific. To support definition of usable pixels, and filtering based on that definition, Planet provides Usable Data Masks along with Usable Data entries in the imagery metadata. For more details on the Usable Data Mask, check out (Clear for Analysis with Planet’s New Usable Data Masks)[https://www.planet.com/pulse/planets-new-usable-data-masks/]. For more information on the Usable Data metadata entries, see (Usable Data in Planet imagery)[https://developers.planet.com/planetschool/usable-data-in-planet-imagery/].

### Orders API

The core decision around using the orders api is which [product bundle](https://developers.planet.com/docs/orders/product-bundles-reference/) to use. This is the starting point for all processing and there are a lot of options. Once the product is determined, the processing steps (aka tools and toolchains) are defined. Finally, the logistics of the delivery of the imagery are ironed out.

#### Product Bundle

To enable time-series analysis, ARD imagery must be processed so that imagery is consistant across days, months, and possibly years. This means correcting for differences in camera sensitivities, the relative location of the sun, and the atmospheric conditions. The Analytic Radiance (`analytic`) product bundle provides imagery corrected for difference in camera sensitivities and location of the sun (as radiance) and can also remove the effect of the sun's spectrum by applying the `reflectanceCoefficient` value given in the imagery metadata. However, the Analytic Surface Reflectance (`analytic_sr`) product bundle removes the effect of atmospheric conditions while also converting to reflectance. Therefore, the Analytic Surface Reflectance is the ideal product for ARD and the one we will use here.

#### Tools and Toolchains

The [Tools and Toolchains](https://developers.planet.com/docs/orders/tools-toolchains/) functionality in the Orders API are the key to seamlessly creating ARD. Through the API, one can define the pre-processing steps for the data before it is delivered. Given proper definition of the tools and toolchains, data that is delivered is analysis-ready.

#### Delivery

There are a few options for delivery that cater to different use cases. Imagery can be downloaded directly or delivered to [cloud storage](https://developers.planet.com/docs/orders/ordering-delivery/#delivery-to-cloud-storage). When imagery is downloaded, the user can poll for when the order is ready or notifications ([e-mail](https://developers.planet.com/docs/orders/ordering-delivery/#email-notification) or [webhooks](https://developers.planet.com/docs/orders/ordering-delivery/#using-webhooks)) can be used. Additionally, the imagery can be delivered as a [zip archive](https://developers.planet.com/docs/orders/ordering-delivery/#zipping-results).

## Use Cases

Now that we have a basic understanding of the Data and Orders APIs, let's put them to use creating ARD for real-world two use cases.

#### ARCHIVE: Use Case 2

The second use case we will cover is: *As a tropical forest monitoring analyst, I'd like to create an imagery pipeline that delivers analysis-ready imagery over a [LARGE] protected area so I can create simple change detection (ndvi_xdate - ndvi_ydate) analyses. I'd like to do minimal custom (non-Planet) data processing to get the imagery in a format that supports a change-detection analysis because my technical skills are limited.*

This case is similar to the last use case, but introduces the need for gridding to handle a large area of interest.


### Use Case 1

The first use case we will cover is: *As a software engineer at an ag-tech company, I'd like to be able to order Planet imagery programmatically in a way that enables the data scientist at my organization to create time-series algorithms (e.g. monitoring ndvi curves over time) without further data cleaning and processing.*

For this use case, the area of interest and time range are not specified. The need for no further processing indicates we should specify a strict usable pixel data filter. For time-series analysis the daily coverage of PS satellites is ideal. The requirement for a programmatic interface implies creation of a function. Therefore, we will create a function that takes as inputs the area of interest, time range, and satellite and returns ndvi. For simplicity, we will 


#### Implementation

For this tutorial we will use the [planet python client](https://github.com/planetlabs/planet-client-python) ([documentation](https://planetlabs.github.io/planet-client-python/index.html)). This client simplifies interactions with the various Planet APIs and also includes a command-line interface.

The first step in using the planet python client is initializing the client with the user API key. Each user (or organization) has their own unique API key. Information on finding the API key and running Docker so that the API key is available in the notebooks is given in this repository's [README](https://github.com/planetlabs/notebooks#install-and-use-these-notebooks).

The next step is building functionality for searching the Data API with the client. This consists of building the search query and then running the search. In building this functionality, we will use test information for data ranges and AOIs to test the functionality, but we will build the functions so those pieces can be changed by the end user.

The last step is building functionality for processing and delivery with the Orders API. At the time of the creation of this tutorial (mid-2019), the Orders API functionality best supported (e.g. [documented](https://planetlabs.github.io/planet-client-python/cli/examples.html#orders-examples)) in the command-line interface (CLI). Therefore, we will use the client CLI for this portion of the tutorial. For delivery logistics, we will 

To summarize, these are the steps:
1. Initialize API client
2. Create Data API Search
3. Create Orders API Toolchain

#### Step 1: Initialize API client

In [2]:
import os

from planet import api

In [3]:
# if your Planet API Key is not set as an environment variable, you can paste it below
API_KEY = os.environ.get('PL_API_KEY', 'PASTE_YOUR_KEY_HERE')

client = api.ClientV1(api_key=API_KEY)

#### Step 2: Create Data API Search

In [4]:
import datetime
from pprint import pprint

from planet.api import filters

In [5]:
# define test data for the filter
test_start_date = datetime.datetime(year=2019,month=4,day=1)
test_stop_date = datetime.datetime(year=2019,month=5,day=1)

# iowa crops aoi
test_aoi_geom = {
    "type": "Polygon",
    "coordinates": [
        [
            [-93.299129, 42.699599],
            [-93.299674, 42.812757],
            [-93.288436, 42.861921],
            [-93.265332, 42.924817],
            [-92.993873, 42.925124],
            [-92.993888, 42.773637],
            [-92.998396, 42.754529],
            [-93.019154, 42.699988],
            [-93.299129, 42.699599]
        ]
    ]
}

In [6]:
# create an api request from the search specifications
def build_request(aoi_geom, start_date, stop_date):
    '''build a data api search request for clear PSScene4Band imagery'''
    item_type = 'PSScene4Band'
    query = filters.and_filter(
        filters.geom_filter(aoi_geom),
        filters.range_filter('clear_percent', gte=90),
        filters.date_range('acquired', gt=start_date),
        filters.date_range('acquired', lt=stop_date)
    )
    return filters.build_search_request(query, ['PSScene4Band'])

request = build_request(test_aoi_geom, test_start_date, test_stop_date)
print(request)

{'item_types': ['PSScene4Band'], 'filter': {'type': 'AndFilter', 'config': ({'field_name': 'geometry', 'type': 'GeometryFilter', 'config': {'type': 'Polygon', 'coordinates': [[[-93.299129, 42.699599], [-93.299674, 42.812757], [-93.288436, 42.861921], [-93.265332, 42.924817], [-92.993873, 42.925124], [-92.993888, 42.773637], [-92.998396, 42.754529], [-93.019154, 42.699988], [-93.299129, 42.699599]]]}}, {'field_name': 'clear_percent', 'type': 'RangeFilter', 'config': {'gte': 90}}, {'field_name': 'acquired', 'type': 'DateRangeFilter', 'config': {'gt': '2019-04-01T00:00:00Z'}}, {'field_name': 'acquired', 'type': 'DateRangeFilter', 'config': {'lt': '2019-05-01T00:00:00Z'}})}}


In [7]:
# search the data api
def search_data_api(request, client, limit=500):
    result = client.quick_search(request)
    
    # this returns a generator
    return result.items_iter(limit=limit)

items = list(search_data_api(request, client))
print(len(items))

80


In [8]:
# check out an item just for fun
pprint(items[0])

{'_links': {'_self': 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20190415_170304_85_1068',
            'assets': 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20190415_170304_85_1068/assets/',
            'thumbnail': 'https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20190415_170304_85_1068/thumb'},
 '_permissions': ['assets.udm:download',
                  'assets.analytic:download',
                  'assets.analytic_xml:download',
                  'assets.analytic_dn:download',
                  'assets.analytic_dn_xml:download',
                  'assets.basic_analytic:download',
                  'assets.basic_analytic_rpc:download',
                  'assets.basic_analytic_dn:download',
                  'assets.basic_analytic_dn_rpc:download',
                  'assets.basic_analytic_xml:download',
                  'assets.basic_analytic_dn_xml:download',
                  'assets.basic_analytic_dn_nitf:download',
               

#### Step 3: TESTING ORDERS Python

#### Step 3: Create Orders API Toolchain

In [24]:
# work with just a subset of the items in the interest of bandwidth
test_items = items[:4]

In [25]:
# filter to item ids then format the ids for use with the CLI
ids = ','.join([i['id'] for i in test_items])
ids

'20190415_170304_85_1068,20190415_170302_79_1068,20190426_170322_82_105c,20190426_170320_32_105c'

In [26]:
# specify the psscene4band surface reflectance product
item_type = 'psscene4band'
bundle = 'analytic_sr'

In [27]:
# zip up each bundle
# this was originally going to be per-order, but it looks like per-order isn't working right now
# https://github.com/planetlabs/planet-client-python/issues/216
zip = 'bundle'

In [28]:
import json

In [48]:
# specify tools

# clip to AOI
clip_tool = {'clip': {'aoi': test_aoi_geom}}

# convert to NDVI
bandmath_tool = {'bandmath': {
    "pixel_type": "32R",
    "b1": "(b4 - b3) / (b4+b3)"
}}

tools = [clip_tool, bandmath_tool]
pprint(tools)
tools_file = 'tools.json'
with open(tools_file, 'w') as dst:
    dst.write(json.dumps(tools))

[{'clip': {'aoi': {'coordinates': [[[-93.299129, 42.699599],
                                    [-93.299674, 42.812757],
                                    [-93.288436, 42.861921],
                                    [-93.265332, 42.924817],
                                    [-92.993873, 42.925124],
                                    [-92.993888, 42.773637],
                                    [-92.998396, 42.754529],
                                    [-93.019154, 42.699988],
                                    [-93.299129, 42.699599]]],
                   'type': 'Polygon'}}},
 {'bandmath': {'b1': '(b4 - b3) / (b4+b3)', 'pixel_type': '32R'}}]


In [49]:
# specify a name
name = 'tutorial_order'

In [61]:
!echo planet orders create \
    --id $ids \
    --item-type $item_type \
    --bundle $bundle \
    --zip $zip \
    --tools $tools_file \
    --name $name

planet orders create --id 20190415_170304_85_1068,20190415_170302_79_1068,20190426_170322_82_105c,20190426_170320_32_105c --item-type psscene4band --bundle analytic_sr --zip order --tools tools.json --name tutorial_order


In [52]:
!planet orders create \
    --id $ids \
    --item-type $item_type \
    --bundle $bundle \
    --zip $zip \
    --tools $tools_file \
    --name $name

{
  "_links": {
    "_self": "https://api.planet.com/compute/ops/orders/v2/2eb5f55e-d260-4ee5-b1b5-7d32ff389dc6"
  },
  "created_on": "2020-05-28T23:46:22.644Z",
  "delivery": {
    "archive_filename": "{{name}}_{{order_id}}.zip",
    "archive_type": "zip"
  },
  "error_hints": [],
  "id": "2eb5f55e-d260-4ee5-b1b5-7d32ff389dc6",
  "last_message": "Preparing order",
  "last_modified": "2020-05-28T23:46:22.644Z",
  "name": "tutorial_order",
  "notifications": {},
  "products": [
    {
      "item_ids": [
        "20190426_170320_32_105c",
        "20190415_170304_85_1068",
        "20190415_170302_79_1068",
        "20190426_170322_82_105c"
      ],
      "item_type": "PSScene4Band",
      "product_bundle": "analytic_sr"
    }
  ],
  "state": "queued",
  "tools": [
    {
      "clip": {
        "aoi": {
          "coordinates": [
            [
              [
                -93.299129,
                42.699599
              ],
              [
                -93.299674,
               

#### Step 4: Download Order

In [55]:
# order id pulled from response
order_id = '2eb5f55e-d260-4ee5-b1b5-7d32ff389dc6'

In [56]:
# check on order status
!planet orders get $order_id

{
  "_links": {
    "_self": "https://api.planet.com/compute/ops/orders/v2/2eb5f55e-d260-4ee5-b1b5-7d32ff389dc6",
    "results": [
      {
        "delivery": "success",
        "expires_at": "2020-05-29T23:52:19.210Z",
        "location": "https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE1OTA3OTYzMzksInN1YiI6IlhsWnBZSjdBTW53bWlnUHN0THFPbS9vNkpkSDJ0OWF4K2JoK0FPcW5nVFNnajRZdDRrRTl1allqV1h2SHROQ1FhMjFuMnBydkc4OU55OXp6T242REVBPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS8yZWI1ZjU1ZS1kMjYwLTRlZTUtYjFiNS03ZDMyZmYzODlkYzYvUFNTY2VuZTRCYW5kXzIwMTkwNDI2XzE3MDMyMF8zMl8xMDVjX2FuYWx5dGljX3NyXzJlYjVmNTVlLWQyNjAtNGVlNS1iMWI1LTdkMzJmZjM4OWRjNi56aXA_RXhwaXJlcz0xNTkwNzk2MzM5XHUwMDI2R29vZ2xlQWNjZXNzSWQ9Y29tcHV0ZS1nY3Mtc3ZjYWNjJTQwcGxhbmV0LWNvbXB1dGUtcHJvZC5pYW0uZ3NlcnZpY2VhY2NvdW50LmN

In [57]:
!planet orders download $order_id

activating: 0            complete: 0              downloaded: 0.0MB             
downloading: 0           elapsed: 0               paging: False                 
pending: 0                                                                      

activating: 0            complete: 0              downloaded: 0.0MB             
downloading: 0           elapsed: 0               paging: False                 
pending: 0                                                                      
2020-05-28 23:59:30,850 - FuturesSession - WARNING - `background_callback` is
deprecated and will be removed in 1.0, use `hooks` instead
activating: 0            complete: 0              downloaded: 0.0MB             
downloading: 0           elapsed: 0               paging: False                 
pending: 0                                                                      
2020-05-28 23:59:30,850 - FuturesSession - WARNING - `background_callback` is
deprecated and will be removed in 1.0, use `hooks` inst

In [60]:
!ls

analysis_ready_data_tutorial.ipynb
manifest.json
PSScene4Band_20190415_170302_79_1068_analytic_sr_2eb5f55e-d260-4ee5-b1b5-7d32ff389dc6.zip
PSScene4Band_20190415_170304_85_1068_analytic_sr_2eb5f55e-d260-4ee5-b1b5-7d32ff389dc6.zip
PSScene4Band_20190426_170320_32_105c_analytic_sr_2eb5f55e-d260-4ee5-b1b5-7d32ff389dc6.zip
PSScene4Band_20190426_170322_82_105c_analytic_sr_2eb5f55e-d260-4ee5-b1b5-7d32ff389dc6.zip
tools.json


In [62]:
!cat manifest.json

{"name":"","files":[{"path":"PSScene4Band_20190426_170322_82_105c_analytic_sr_2eb5f55e-d260-4ee5-b1b5-7d32ff389dc6.zip","media_type":"application/zip","size":127864659,"digests":{"md5":"7a765d80bb25814ce1a936158e122e4c","sha256":"2730b023572baee45ccfede9cfd2485b4cb0436b41da08f5958eb70fe9d778d8"},"annotations":{}},{"path":"PSScene4Band_20190426_170320_32_105c_analytic_sr_2eb5f55e-d260-4ee5-b1b5-7d32ff389dc6.zip","media_type":"application/zip","size":101456331,"digests":{"md5":"c634da295187067125a21ce62f516e62","sha256":"24d550356e1f2d5cce586b55cb458bc3a05f1856936a09a9e2a0990186a0f441"},"annotations":{}},{"path":"PSScene4Band_20190415_170302_79_1068_analytic_sr_2eb5f55e-d260-4ee5-b1b5-7d32ff389dc6.zip","media_type":"application/zip","size":75301112,"digests":{"md5":"9d8270c980d98a3048f53d32edb3d4e4","sha256":"3474efc28794d8362baeda1f95bc124da7636bf4c596464a600e4f109fc50a03"},"annotations":{}},{"path":"PSScene4Band_20190415_170304_85_1068_analytic_sr_2eb5f55e-d260-4ee5-b1b5-7d32ff389dc6.z

#### Bonus Step: Checking out the Imagery


#### Try using Python

ok, so CLI is a PAIN to work with in regard to getting order ids, image files, unzipping and viewing.

Lets *try* to use the python api to interact with orders

In [76]:
# check on orders already submitted
orders = client.get_orders().get()['orders']

# lets get an id!
orders[0]['id']

'2eb5f55e-d260-4ee5-b1b5-7d32ff389dc6'